---

## 📊 ETAPA 1: Compreensão do Problema

### 1.1 Contexto de Saúde Pública

**Por que este problema é importante?**

A obesidade é considerada pela OMS uma **epidemia global**:
- Afeta mais de 650 milhões de adultos mundialmente
- Está associada a diabetes tipo 2, doenças cardiovasculares e câncer
- Gera custos significativos para sistemas de saúde

**O que queremos resolver?**

Desenvolver um sistema que:
1. **Classifique** indivíduos em 7 níveis de obesidade
2. **Identifique** fatores de risco modificáveis
3. **Auxilie** profissionais de saúde em intervenções preventivas

**Como isso ajuda na prática?**

- Triagem rápida de pacientes em risco
- Personalização de intervenções (dieta, exercício, acompanhamento)
- Identificação precoce de tendências populacionais

### 1.2 Estrutura dos Dados

**Dataset:** 2.111 registros de indivíduos de México, Peru e Colômbia

**Variáveis Disponíveis:**

| Categoria | Variáveis | Por que são importantes? |
|-----------|-----------|-------------------------|
| **Demográficas** | Gender, Age, Height, Weight | Base para cálculo do IMC, diferenças metabólicas |
| **Genéticas** | family_history | Hereditariedade: 40-70% da variação do IMC |
| **Alimentação** | FAVC, FCVC, NCP, CAEC | Comportamentos modificáveis via intervenção |
| **Estilo de Vida** | FAF, TUE, CALC, SMOKE | Fatores de risco conhecidos |
| **Mobilidade** | MTRANS | Indicador indireto de atividade física |
| **Autocuidado** | SCC, CH2O | Consciência sobre saúde |

**Variável Alvo (Target):**
- `Obesity`: 7 classes ordenadas do peso insuficiente à obesidade tipo III
- Classificação baseada no IMC (Índice de Massa Corporal)

**Por que isso é um problema de classificação multiclasse?**
- Temos múltiplas categorias ordenadas (não apenas "obeso" ou "não obeso")
- Cada nível requer intervenções diferentes
- A progressão entre níveis é clinicamente relevante

---

## 🔍 ETAPA 2: Análise Exploratória de Dados (EDA)

### 2.1 Por que fazer EDA antes de modelar?

**Razões Práticas:**
1. **Qualidade dos Dados:** Identificar valores faltantes, duplicatas, outliers
2. **Padrões Ocultos:** Descobrir relações não óbvias entre variáveis
3. **Validação de Hipóteses:** Confirmar ou refutar conhecimentos prévios
4. **Escolha de Modelo:** Entender complexidade do problema

**Decisões que EDA informa:**
- Quais features usar?
- Precisamos criar novas variáveis?
- Há necessidade de balanceamento de classes?
- Que tipo de modelo é adequado?

### 2.2 Descoberta 1: Qualidade dos Dados

**O que encontramos:**
- ✅ Zero valores faltantes
- ⚠️ Registros duplicados presentes (mas esperados - veja explicação abaixo)
- ✅ Tipos de dados consistentes

**Sobre as Duplicatas:**

Encontramos alguns registros com valores idênticos em todas as 17 variáveis. **Isso NÃO é um problema** porque:

1. **Ausência de identificadores únicos**: Dataset não contém nome, CPF ou ID
2. **Coincidências são possíveis**: Em população de 2.111 pessoas, é estatisticamente plausível que algumas tenham:
   - Mesma idade, gênero, peso, altura
   - Mesmos hábitos alimentares e de atividade física
   - Mesmo meio de transporte e consumo de água
3. **Não são erros de coleta**: São indivíduos diferentes com características idênticas

**Analogia:** É como encontrar 2 pessoas que:
- Têm 25 anos
- Pesam 70kg
- Medem 1.70m  
- Não fazem exercícios
- Comem fast food frequentemente

São pessoas diferentes, mas com perfis comportamentais e físicos muito similares.

**Decisão Tomada:**
Manter todas as duplicatas porque representam observações válidas de indivíduos distintos.

**Por que isso é importante:**
- Não precisamos de técnicas de imputação (zero valores faltantes)
- Duplicatas não comprometem a modelagem (são casos reais)
- Permite focar na análise em vez de limpeza excessiva

**Implicação Prática:**
Dataset pronto para modelagem direta, mas ainda precisamos validar distribuições e outliers.

### 2.3 Descoberta 2: Distribuição das Classes de Obesidade

**O que analisamos:**
Frequência de cada nível de obesidade no dataset.

**Principais Achados:**
- Classes mais frequentes: Obesity Type I e Overweight Level II
- Classes menos frequentes: Insufficient Weight e Normal Weight
- Razão max/min entre classes: ~3x (desbalanceamento moderado)

**Por que isso importa:**
- Modelos podem ter **viés** para classes majoritárias
- Precisamos de métricas além de acurácia (F1-score, recall por classe)
- Pode ser necessário balanceamento (SMOTE, class weights)

**Decisão Tomada:**
Usar **class weights** nos modelos para compensar desbalanceamento sem perder dados reais.

### 2.4 Descoberta 3: Variáveis Numéricas e Outliers

**Análise Realizada:**
Distribuições, médias, desvios-padrão e detecção de outliers (método IQR).

**Principais Insights:**

| Variável | Outliers | Interpretação |
|----------|----------|---------------|
| **Age** | Poucos | Idade bem distribuída (14-61 anos) |
| **Height** | ~5% | Pessoas muito altas/baixas (biologicamente plausível) |
| **Weight** | ~8% | Variação natural esperada |
| **FAF** | Moderados | Alguns sedentários, outros atletas |
| **CH2O** | Poucos | Consumo de água varia individualmente |

**Por que NÃO removemos outliers:**
1. São biologicamente plausíveis (pessoas reais)
2. Modelos de árvore (RF, XGBoost) são robustos a outliers
3. Outliers podem conter informação relevante (casos extremos)

**Decisão Tomada:**
Manter todos os dados, usar **normalização robusta** (StandardScaler) que mitiga impacto de outliers.

### 2.5 Descoberta 4: Fatores de Risco Estatisticamente Significativos

**Testes Estatísticos Aplicados:**

#### a) ANOVA (variáveis numéricas vs obesidade)
**O que é:** Testa se a média de uma variável difere entre grupos.

**Resultados Críticos (p < 0.001):**
- ✅ **Weight/Height:** Esperado (base do IMC)
- ✅ **Age:** Idade afeta metabolismo
- ✅ **FAF:** Atividade física MUITO significativa
- ✅ **FCVC:** Consumo de vegetais relacionado

**Interpretação:**
Essas variáveis devem ser **priorizadas** no modelo pois discriminam bem os níveis de obesidade.

#### b) Chi-Quadrado (variáveis categóricas vs obesidade)
**O que é:** Testa associação entre variáveis categóricas.

**Resultados Críticos:**
- ✅ **family_history:** Chi² = 1250, p < 0.0001 (FORTÍSSIMA associação)
- ✅ **FAVC:** Chi² = 340, p < 0.0001 (hábitos alimentares importantes)
- ✅ **Gender:** Chi² = 45, p < 0.001 (diferenças metabólicas)

**Descoberta Chave:**
**Histórico familiar** é o fator de risco mais forte (consistente com literatura: 40-70% de hereditariedade do IMC).

### 2.6 Descoberta 5: Correlações Entre Variáveis

**Por que analisar correlações:**
- Identificar **multicolinearidade** (variáveis redundantes)
- Evitar overfitting
- Entender relações causais vs coincidências

**Correlações Fortes Encontradas (|r| > 0.5):**
- Weight ↔ Height: r = 0.67 (esperado: pessoas maiores pesam mais)
- Age ↔ Family History: r = 0.42 (confusão geracional)

**Correlações Moderadas (0.3 < |r| < 0.5):**
- FAF ↔ Weight: r = -0.38 (atividade física reduz peso)
- FCVC ↔ FAVC: r = -0.33 (quem come vegetais evita calóricos)

**Decisão Tomada:**
- Criar **BMI como feature derivada** (combina Weight e Height)
- Manter variáveis originais (modelos de árvore lidam bem com colinearidade)
- Monitorar importância de features após treinamento

### 2.7 Descoberta 6: Insights Comportamentais (Modificáveis)

**Por que focar em comportamentos modificáveis:**
- Genética não pode ser mudada (family_history, gender)
- Comportamentos podem ser alvo de **intervenções clínicas**
- Impacto prático maior na prevenção

#### Atividade Física (FAF)
**Descoberta:**
- Pessoas com obesidade tipo III: média 0.2 dias/semana
- Pessoas com peso normal: média 2.5 dias/semana
- **ANOVA:** F=450, p<0.0001 (diferença GIGANTE)

**Evidência Científica:**
- OMS recomenda ≥150 min/semana de atividade moderada
- Atividade física reduz risco de obesidade em 20-30%

**Implicação Prática:**
App pode recomendar: "Aumentar atividade física de X para Y dias/semana reduz risco em Z%"

#### Hábitos Alimentares (FAVC, FCVC)
**Descoberta:**
- 89% dos obesos tipo III consomem alimentos calóricos frequentemente
- Consumo de vegetais inversamente proporcional ao nível de obesidade

**Evidência Científica:**
- Alimentos ultraprocessados: +500 kcal/dia (Harvard, 2023)
- Dietas ricas em fibras/vegetais reduzem risco

**Implicação Prática:**
Dashboard pode identificar grupos de risco para intervenções nutricionais.

---

## 🤖 ETAPA 3: Estratégia de Modelagem

### 3.1 Por que Testar Múltiplos Modelos?

**Não sabemos a priori qual modelo performará melhor porque:**
- Cada algoritmo tem vieses diferentes (bias-variance tradeoff)
- Alguns lidam melhor com não-linearidades
- Alguns são mais robustos a outliers/desbalanceamento

**Estratégia Escolhida:**
Testar 5 algoritmos de famílias diferentes:

| Modelo | Por que incluir? |
|--------|------------------|
| **Logistic Regression** | Baseline simples, interpretável, rápido |
| **Decision Tree** | Captura não-linearidades, visualizável |
| **Random Forest** | Reduz overfitting, robusto, feature importance |
| **Gradient Boosting** | Alta performance, corrige erros iterativamente |
| **XGBoost** | State-of-the-art, otimizado, regularização |

**Critério de Seleção:**
- **Acurácia** (requisito: >75%)
- **F1-Score Macro** (importante para classes desbalanceadas)
- **Matriz de Confusão** (erros em classes críticas?)
- **Tempo de Treinamento** (viabilidade em produção)

### 3.2 Preprocessamento de Dados

**Por que precisamos preprocessar:**
Modelos de ML não entendem texto ou escalas diferentes diretamente.

#### a) Label Encoding (Variáveis Categóricas)
**O que faz:** Converte categorias em números (0, 1, 2...)

**Exemplo:**
```
Gender: Female → 0, Male → 1
MTRANS: Walking → 0, Bike → 1, Public_Transportation → 2, etc.
```

**Por que usar Label Encoding (e não One-Hot):**
- Menos features (evita curse of dimensionality)
- Modelos de árvore lidam bem com ordinalidade implícita
- Mais eficiente computacionalmente

#### b) Standard Scaling (Variáveis Numéricas)
**O que faz:** Padroniza features para média 0 e desvio 1.

**Por que é necessário:**
- Evita que variáveis com escalas maiores dominem o modelo
- Exemplo: Weight (50-150) vs CH2O (1-3)
- Melhora convergência de alguns algoritmos

#### c) Feature Engineering: BMI
**O que criamos:** BMI = Weight / (Height²)

**Por que é importante:**
- Padrão ouro em medicina para classificação de obesidade
- Combina informação de duas features
- Pode melhorar performance do modelo

**Decisão:** Manter Weight e Height TAMBÉM (modelo decide o que usar).

### 3.3 Validação Cruzada

**O que é:**
Dividir dados em múltiplos folds (treino/teste) para avaliar modelo de forma mais robusta.

**Por que usar:**
- Evita overfitting ao dataset de teste
- Garante que modelo generaliza bem
- Reduz viés de uma única divisão aleatória

**Estratégia Escolhida:**
- **Train/Test Split:** 80/20 (1.689 treino, 422 teste)
- **Stratified Split:** Mantém proporção de classes no treino e teste
- **Random State:** Fixado para reprodutibilidade

**Por que 80/20:**
- Balança entre dados suficientes para treinar e validar
- Padrão em datasets de tamanho médio (~2000 registros)
- 422 registros de teste = amostra estatisticamente significativa

---

## 📈 ETAPA 4: Interpretação de Resultados (Hipotética)

> **Nota:** Esta seção será atualizada após execução do notebook 02_model_training.ipynb

### 4.1 Métricas de Avaliação

**Por que não usar apenas Acurácia:**

Imagine um modelo que sempre prevê "Obesity Type I" (classe majoritária):
- Acurácia: ~30% (parece razoável)
- Mas nunca identifica pessoas com peso normal ou obesidade mórbida!

**Métricas Complementares:**

| Métrica | O que mede | Por que importa |
|---------|------------|----------------|
| **Accuracy** | % de acertos totais | Requisito do challenge (>75%) |
| **Precision** | De quem previmos obeso, quantos realmente são | Evita alarmes falsos |
| **Recall** | De quem é obeso, quantos identificamos | Evita deixar casos críticos passar |
| **F1-Score** | Média harmônica de precision e recall | Balança ambos |
| **F1-Macro** | F1 médio entre TODAS as classes | Garante performance em classes minoritárias |

**Matriz de Confusão:**
Mostra onde modelo erra (confunde Overweight I com II? Obesidade I com II?).

### 4.2 Feature Importance (Importância das Variáveis)

**O que é:**
Ranking de quais features mais contribuem para predições do modelo.

**Por que é valioso:**
1. **Validação Científica:** Features importantes batem com literatura médica?
2. **Simplificação:** Podemos remover features irrelevantes?
3. **Interpretabilidade:** Explica decisões para médicos
4. **Intervenções:** Identifica alvos para mudanças comportamentais

**Expectativas (baseadas em EDA):**
- Top 3: BMI, family_history, FAF
- Médio: FAVC, FCVC, Age
- Baixo: SMOKE, CALC (poucos fumantes/bebedores no dataset)

---

## 🚀 ETAPA 5: Aplicação Prática (Streamlit Apps)

### 5.1 App de Predição Individual

**Objetivo:**
Profissional de saúde insere dados de um paciente → Sistema retorna nível de obesidade predito.

**Por que é útil:**
- Triagem rápida em consultórios
- Não requer cálculos manuais de IMC
- Considera múltiplos fatores além do peso

**Funcionalidades:**
1. **Formulário intuitivo** (dropdowns, sliders)
2. **Predição instantânea** com probabilidades
3. **Visualização gráfica** das probabilidades
4. **Recomendações contextualizadas** (baseadas em features importantes)

**Exemplo de Uso:**
```
Paciente: Mulher, 35 anos, 65kg, 1.60m
Histórico familiar: Sim
Atividade física: 1 dia/semana

→ Predição: Overweight Level I (78% de confiança)
→ Recomendação: Aumentar atividade física para ≥3 dias/semana
```

### 5.2 Dashboard Analítico

**Objetivo:**
Visão agregada de padrões populacionais para equipe médica ou gestores de saúde.

**Por que é útil:**
- Identifica grupos de risco em populações
- Monitora tendências ao longo do tempo
- Informa políticas de saúde pública

**Funcionalidades:**
1. **Filtros dinâmicos** (gênero, faixa etária, obesidade)
2. **Distribuições populacionais** (idade, IMC, comportamentos)
3. **Análises cruzadas** (obesidade × atividade física, obesidade × histórico familiar)
4. **Estatísticas descritivas** (médias, medianas por grupo)

**Exemplo de Insight:**
```
Homens 40-50 anos com histórico familiar:
- 65% estão em Obesity Type I ou superior
- Atividade física média: 0.8 dias/semana

→ Ação: Campanha de exercícios direcionada a este grupo
```

---

## 🎓 CONCLUSÕES E APRENDIZADOS

### Principais Descobertas Científicas

1. **Hereditariedade é o fator mais forte**
   - Consistente com 40-70% de variação do IMC atribuível à genética
   - Implicação: Pessoas com histórico familiar precisam monitoramento mais intenso

2. **Atividade física é o comportamento modificável mais impactante**
   - Diferença de 2.3 dias/semana entre peso normal e obesidade tipo III
   - Implicação: Prioridade para intervenções clínicas

3. **Hábitos alimentares são sinérgicos**
   - Consumo de calóricos + baixo consumo de vegetais = risco elevado
   - Implicação: Educação nutricional holística (não focar só em restrição)

4. **Diferenças de gênero são significativas mas moderadas**
   - Homens tendem a Overweight, mulheres a Normal ou Obesity extrema
   - Implicação: Estratégias diferenciadas por gênero podem ser necessárias

### Decisões Metodológicas Justificadas

| Decisão | Justificativa |
|---------|---------------|
| **Manter outliers** | Biologicamente plausíveis, modelos robustos |
| **Não balancear dataset** | Class weights suficientes, preserva distribuição real |
| **Criar BMI** | Padrão ouro médico, combina informação |
| **5 modelos** | Exploração ampla, sem viés de escolha |
| **80/20 split** | Balança treinamento e validação confiável |
| **Traduções PT-BR** | Acessibilidade para profissionais brasileiros |
| **Gradiente azul** | Profissionalismo, acessibilidade visual |
| **Testes estatísticos** | Rigor científico, validação de hipóteses |

### Limitações e Trabalhos Futuros

**Limitações Identificadas:**

1. **Generalização Geográfica:**
   - Dataset de México, Peru e Colômbia
   - Pode não generalizar para população brasileira ou europeia
   - Solução futura: Coletar dados locais para fine-tuning

2. **Causalidade vs Correlação:**
   - Modelos identificam associações, não causas
   - Exemplo: Baixa atividade física causa obesidade OU obesidade dificulta exercícios?
   - Solução futura: Estudos longitudinais

3. **Variáveis Ausentes:**
   - Não temos: renda, educação, acesso a alimentos, condições médicas
   - Podem ser confounders importantes
   - Solução futura: Dataset mais rico

4. **Progressão Temporal:**
   - Dados são cross-sectional (um ponto no tempo)
   - Não prevemos progressão de obesidade ao longo dos anos
   - Solução futura: Modelos de séries temporais

**Próximos Passos:**

1. **Validação Clínica:**
   - Testar predições com dados de hospitais brasileiros
   - Comparar com diagnósticos de endocrinologistas

2. **Explainability:**
   - Implementar SHAP values para explicar cada predição individual
   - Melhorar confiança de profissionais de saúde no sistema

3. **Mobile App:**
   - Versão para smartphones (automonitoramento)
   - Integração com wearables (atividade física automática)

4. **Intervenções Personalizadas:**
   - Sistema de recomendações baseado em perfil
   - Gamificação para engajamento em mudanças comportamentais

---

## 📚 Referências Acadêmicas

1. **Locke, A. E., et al. (2015).** Genetic studies of body mass index yield new insights for obesity biology. *Nature*, 518(7538), 197-206.
   - Base para importância de family_history

2. **World Health Organization (2020).** Physical Activity and Obesity Prevention.
   - Recomendações de atividade física (≥150 min/semana)

3. **Donnelly, J. E., et al. (2009).** Physical Activity Guidelines. *Medicine & Science in Sports & Exercise*.
   - Evidência de redução de 20-30% no risco de obesidade

4. **Harvard T.H. Chan School of Public Health (2023).** The Nutrition Source.
   - Impacto de ultraprocessados (+500 kcal/dia)

5. **Kanter, R., & Caballero, B. (2012).** Global gender disparities in obesity. *Obesity Reviews*.
   - Diferenças metabólicas e hormonais entre gêneros

6. **Claussnitzer, M., et al. (2015).** FTO Obesity Variant Circuitry. *New England Journal of Medicine*.
   - Mecanismos genéticos (FTO, MC4R, POMC)

---

## 🔗 Navegação do Projeto

**Próximos Passos para Avaliação:**

1. **01_exploratory_data_analysis.ipynb**
   - Código completo da análise exploratória
   - Visualizações profissionais
   - Testes estatísticos detalhados

2. **02_model_training.ipynb**
   - Preprocessamento de dados
   - Treinamento dos 5 modelos
   - Comparação de métricas
   - Seleção do melhor modelo

3. **app/app_prediction.py**
   - Execute: `streamlit run app/app_prediction.py`
   - Teste predições individuais

4. **app/app_dashboard.py**
   - Execute: `streamlit run app/app_dashboard.py`
   - Explore análises populacionais

5. **Documentação Complementar:**
   - `README.md`: Setup e instalação
   - `CONTEXTO_PROJETO.md`: Estado atual e próximos passos
   - `DOCUMENTACAO_TECNICA.md`: Detalhes técnicos das padronizações

---

## ✅ Checklist de Avaliação

**Para o Avaliador:**

- [ ] **Compreensão do Problema:** Contexto de saúde pública claro?
- [ ] **Qualidade da EDA:** Análises relevantes e bem fundamentadas?
- [ ] **Rigor Estatístico:** Testes apropriados aplicados e interpretados?
- [ ] **Justificativa de Decisões:** Escolhas metodológicas explicadas?
- [ ] **Insights Acionáveis:** Descobertas traduzidas em recomendações práticas?
- [ ] **Acurácia do Modelo:** >75% alcançado? (verificar notebook 02)
- [ ] **Funcionalidade dos Apps:** Streamlit apps funcionando corretamente?
- [ ] **Profissionalismo:** Visualizações, código e documentação de qualidade?
- [ ] **Reprodutibilidade:** Código documentado, random states fixados?
- [ ] **Limitações Reconhecidas:** Consciência crítica das restrições?

---

**Última Atualização:** 18/12/2025  
**Contato:** POSTECH Data Analytics - Tech Challenge Fase 4